In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os

import shutil

try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

%cd /content/drive/My Drive/phishing1


Mounted at /content/drive
/content/drive/My Drive/phishing1


In [ ]:
#phishing Dataset
folder_dataset_test = datasets.ImageFolder(root="/content/drive/My Drive/.../")
#training Dataset
folder_dataset = datasets.ImageFolder(root="/content/drive/My Drive/.../")
#eval Dataset
folder_dataset_test_original = datasets.ImageFolder(root="/content/drive/My Drive/.../")

# pair_list_file = '/content/drive/My Drive/phishing1/combined.txt'
pair_list_file = '/content/drive/My Drive/.../amazon.txt'
#   highvarietysmalldata
#   trusted_list_and_phishing
#   trusted_list_1img
#training config

learningrate = 0.0001
epoch = 30
batch = 16
architecture = ""

#txt file
txtfile = f'{epoch}epoch_adam{learningrate:.5f}_batch{batch}_{architecture}.txt'


namemodel2save = f'/content/drive/My Drive/.../{epoch}epoch_adam{learningrate:.5f}_batch{batch}_{architecture}.pth'




In [ ]:
# Creating some helper functions
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
   # plt.figure(figsize=(224,224))
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})

    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:
#wird benötigt für Testdatensatz
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform

    def __getitem__(self,index):
        img0_tuple = self.imageFolderDataset.imgs[index]
        # print(img0_tuple, "img tuple", "\n")
        # print(index, "INDEX", "\n")
        #Choose a random index for the second image
        img1_index = random.randint(0, len(self.imageFolderDataset) - 1)
        #print(img1_index, "img 1 INDEX", "\n")
        #Ensure that the two images are not the same
        while img1_index == index:
            img1_index = random.randint(0, len(self.imageFolderDataset) - 1)

        img1_tuple = self.imageFolderDataset.imgs[img1_index]

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert('RGB')
        img1 = img1.convert('RGB')

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [ ]:
class SiameseNetworkDataset1(Dataset):
    def __init__(self, pairs_file_path=None, imageFolderDataset=None, transform=None):
        if pairs_file_path is not None:
            self.pairs = []
            with open(pairs_file_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    img0_path, img1_path, label = line.strip().split(',')
                    self.pairs.append((img0_path, img1_path, int(label)))
        elif imageFolderDataset is not None:
            self.imageFolderDataset = imageFolderDataset
        else:
            raise ValueError("Either 'pairs_file_path' or 'imageFolderDataset' must be provided.")

        self.transform = transform

    def __getitem__(self, index):
        if hasattr(self, 'pairs'):
            img0_path, img1_path, label = self.pairs[index]
            img0 = Image.open(img0_path).convert('RGB')
            img1 = Image.open(img1_path).convert('RGB')
        else:
            img0_tuple = self.imageFolderDataset.imgs[index]
            img1_index = random.randint(0, len(self.imageFolderDataset) - 1)
            while img1_index == index:
                img1_index = random.randint(0, len(self.imageFolderDataset) - 1)
            img1_tuple = self.imageFolderDataset.imgs[img1_index]
            img0 = Image.open(img0_tuple[0]).convert('RGB')
            img1 = Image.open(img1_tuple[0]).convert('RGB')

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        return img1, img0, torch.tensor([label], dtype=torch.float32)

    def __len__(self):
        if hasattr(self, 'pairs'):
            return len(self.pairs)
        else:
            return len(self.imageFolderDataset.imgs)

In [ ]:
# Resize the images and transform to tensors
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
transformation = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                    ])

            # tv.transforms.Resize((224, 224)),
            # tv.transforms.RandomCrop((224, 224)),
            # tv.transforms.RandomHorizontalFlip(),
            # tv.transforms.ToTensor(),
            # tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),



In [ ]:
# pair_list_file = '/content/drive/My Drive/phishing1/combined.txt'
pair_list_file = '/content/drive/My Drive/phishing1/amazon.txt'
siamese_dataset = SiameseNetworkDataset1(pair_list_file, transform=transformation)
#

In [ ]:
# Create a simple dataloader just for simple visualization
vis_dataloader = DataLoader(siamese_dataset,
                        shuffle=False,
                        num_workers=2,
                        batch_size=8)

# Extract one batch
example_batch = next(iter(vis_dataloader))

# Example batch is a list containing 2x8 images, indexes 0 and 1, an also the label
# If the label is 1, it means that it is not the same person, label is 0, same person in both images
concatenated = torch.cat((example_batch[0], example_batch[1]),0)

imshow(torchvision.utils.make_grid(concatenated))
print(example_batch[2].numpy().reshape(-1))
# picture = torchvision.utils.make_grid(example_batch[1])
# extractedInformation = pytesseract.image_to_string(Image.open(example_batch[1]))

In [ ]:
# #create the Siamese Neural Network Datahacker mit Layeranpassung
# class SiameseNetwork(nn.Module):

#     def __init__(self):
#         super(SiameseNetwork, self).__init__()

#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(3, 96, kernel_size=11,stride=4),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(3, stride=2),

#             nn.Conv2d(96, 256, kernel_size=5, stride=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),

#             nn.Conv2d(256, 384, kernel_size=3,stride=1),
#             nn.ReLU(inplace=True)
#         )

#         # Setting up the Fully Connected Layers
#         self.fc1 = nn.Sequential(
#             nn.Linear(31104, 9096),
#             nn.ReLU(inplace=True),

#             nn.Linear(9096, 1024),
#             nn.ReLU(inplace=True),

#             nn.Linear(1024, 256),
#             nn.ReLU(inplace=True),

#             nn.Linear(256,2)
#         )

#     def forward_once(self, x):
#         # This function will be called for both images
#         # It's output is used to determine the similarity
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         # In this function we pass in both images and obtain both vectors
#         # which are returned
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)

#         return output1, output2

In [ ]:
class SiameseNetwork(nn.Module):

    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=10, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.25),  # Beispielwert, du kannst die Dropout-Rate anpassen

            nn.Conv2d(64, 128, kernel_size=7, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(128, 128, kernel_size=4, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2),

            nn.Conv2d(128, 256, kernel_size=4, stride=1),
            nn.ReLU(inplace=True),
        )


        self.fc1 = nn.Sequential(
            nn.Linear(12544, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),  # Dropout layer to prevent overfitting
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),  # Dropout layer to prevent overfitting
            nn.Linear(256, 2)
        )

    def forward_once(self, x):
        output = self.cnn1(x)
        output = output.view(output.size()[0], -1)
        output = self.fc1(output)
        return output

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [ ]:
# #AlexNet
# #https://gist.github.com/bigsnarfdude/f90b7a3bc6965ffc4c5e89e1d6a6c7b0
# #create the Siamese Neural Network
# class SiameseNetwork(nn.Module):

#     def __init__(self):
#         super(SiameseNetwork, self).__init__()

#         # Setting up the Sequential of CNN Layers
#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=10, stride=2, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#         )


#         # Setting up the Fully Connected Layers
#         self.fc1 = nn.Sequential(


#           #(1x9216 and 12544x512)
#           #Ram durch viel zu groß

#              nn.Linear(43264 , 512),
#              nn.ReLU(inplace=True),
#              nn.Linear(512, 256),
#              nn.ReLU(inplace=True),
#              nn.Linear(256, 2)
#         )

#     def forward_once(self, x):
#         # This function will be called for both images
#         # It's output is used to determine the similiarity
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         # In this function we pass in both images and obtain both vectors
#         # which are returned
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)

#         return output1, output2

In [ ]:
# Define the Contrastive Loss Function
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
      # Calculate the euclidian distance and calculate the contrastive loss
      euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

      loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


      return loss_contrastive

In [ ]:
# Load the training dataset
train_dataloader = DataLoader(siamese_dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=batch)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
net = SiameseNetwork().cuda()
criterion = ContrastiveLoss()
#optimizer = optim.Adam(net.parameters(),learningrate)
optimizer = optim.Adam(net.parameters(), learningrate, weight_decay=1e-5)
#net.summary()

In [ ]:
counter = []
loss_history = []
iteration_number= 0

# Iterate throught the epochs
for epoch in range(epoch):

    # Iterate over batches
    for i, (img0, img1, label) in enumerate(train_dataloader, 0):

        # Send the images and labels to CUDA
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        # Zero the gradients
        optimizer.zero_grad()

        # Pass in the two images into the network and obtain two outputs
        output1, output2 = net(img0, img1)

        # Pass the outputs of the networks and label into the loss function
        loss_contrastive = criterion(output1, output2, label)

        # Calculate the backpropagation
        loss_contrastive.backward()

        # Optimize
        optimizer.step()

        # Every 10 batches print out the loss
        if i % 10 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 10

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

show_plot(counter, loss_history)
torch.save(net.state_dict(), namemodel2save)

# Locate the test dataset and load it into the SiameseNetworkDataset

siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transformation)
test_dataloader = DataLoader(siamese_dataset, num_workers=2, batch_size=1, shuffle=False)


# Locate the test dataset and load it into the SiameseNetworkDataset

siamese_dataset_original = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test_original,
                                        transform=transformation)
test_dataloader_original = DataLoader(siamese_dataset_original, num_workers=2, batch_size=1, shuffle=False)

# Grab one image that we are going to test
dataiter = iter(test_dataloader)
dataiter_ori = iter(test_dataloader_original)
file = open(txtfile, 'w')

for trusted1, _, _ in dataiter_ori:
    for x1, _, _ in test_dataloader:

      concatenated = torch.cat((trusted1, x1), 0)

      output1, output2 = net(trusted1.cuda(), x1.cuda())
      euclidean_distance = F.pairwise_distance(output1, output2)
      distance = f' {euclidean_distance.item():.2f}'
      #file.write(str(distance) + '\n')
      file.write(str(distance).replace('.', ',') + '\n')
      imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')

file.close()


